In [1]:

import sys
import os
project_root = os.path.abspath(os.path.join(os.getcwd(), r'C:\Users\Joel\Desktop\Sprints Case Study'))

if project_root not in sys.path:
    sys.path.append(project_root)

from utils.connectors import API_Client
from utils.transformations import remove_invalid_rows, convert_to_dataframe, convert_currency, annual_aggregation
from datetime import datetime
import pandas as pd

base_url = "https://technical-case-platform-engineer.onrender.com"

exchange_rate_endpoint = "exchange-rates"
monthly_date_endpoint = "monthly-data"
annual_data_endpoint = "annual-data"

today = datetime.today()

In [2]:
api_connector = API_Client(base_url)

currency_data = api_connector.get(exchange_rate_endpoint)
value_data = api_connector.get(monthly_date_endpoint)

currency_df = convert_to_dataframe(currency_data)
value_df = convert_to_dataframe(value_data)

In [3]:
cleaned_value_df = remove_invalid_rows(value_df, currency_df)
cleaned_value_df

,timestamp,value,company,currency
1,2025-01-01,6512.17,Solvex Solutions,SEK
2,2024-12-01,5394.51,Solvex Solutions,SEK
3,2024-11-01,3040.15,Solvex Solutions,SEK
4,2024-10-01,6558.26,Solvex Solutions,EUR
5,2024-09-01,5402.61,Solvex Solutions,SEK
...,...,...,...,...
428,2013-06-01,9090.43,VertexSphere,USD
429,2013-05-01,6123.12,VertexSphere,USD
430,2013-04-01,4246.13,VertexSphere,USD
431,2013-03-01,1381.20,VertexSphere,EUR


In [5]:
converted_df = convert_currency(cleaned_value_df, currency_df)
converted_df

,timestamp,value,company,currency
1,2025-01-01,6512.1700,Solvex Solutions,SEK
2,2024-12-01,5394.5100,Solvex Solutions,SEK
3,2024-11-01,3040.1500,Solvex Solutions,SEK
4,2024-10-01,75419.9900,Solvex Solutions,SEK
5,2024-09-01,5402.6100,Solvex Solutions,SEK
...,...,...,...,...
428,2013-06-01,102540.0504,VertexSphere,SEK
429,2013-05-01,69068.7936,VertexSphere,SEK
430,2013-04-01,47896.3464,VertexSphere,SEK
431,2013-03-01,15883.8000,VertexSphere,SEK


In [9]:
monthly_max_values = converted_df.groupby(["timestamp", "currency"])["value"].max().reset_index()

monthly_max_df = pd.merge(
    monthly_max_values, 
    converted_df, 
    on=["timestamp", "currency", "value"], 
    how="left"
)

monthly_max_df.to_csv("monthly-max-values.csv")

In [12]:
nexara_filtered = converted_df[converted_df["company"]=="Nexara Technologies"]
nexara_total = nexara_filtered.groupby(["company", "currency"]).agg(
    value = ("value", "sum")
).reset_index()

nexara_total.to_csv("nexara_total.csv")

In [ ]:
annual_agg = annual_aggregation(converted_df)
annual_agg

,company,year,currency,value
0,Nexara Technologies,2013,SEK,56254.686380
1,Nexara Technologies,2014,SEK,55221.314350
2,Nexara Technologies,2015,SEK,56392.964327
3,Nexara Technologies,2016,SEK,59863.999850
4,Nexara Technologies,2017,SEK,40127.559717
5,Nexara Technologies,2018,SEK,48112.668333
6,Nexara Technologies,2019,SEK,49536.779533
7,Nexara Technologies,2020,SEK,60533.855660
8,Nexara Technologies,2021,SEK,46364.305617
9,Nexara Technologies,2022,SEK,48985.575850
